In [2]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm as tqdm

In [8]:
import stanza

2023-11-30 11:45:17 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-30 11:45:19 INFO: Loading these models for language: en (English):
| Processor | Package          |
--------------------------------
| tokenize  | combined         |
| ner       | ontonotes_charlm |

2023-11-30 11:45:19 INFO: Using device: cpu
2023-11-30 11:45:19 INFO: Loading: tokenize
2023-11-30 11:45:20 INFO: Loading: ner
2023-11-30 11:45:21 INFO: Done loading processors!


entity: Chris Manning	type: PERSON
entity: Stanford University	type: ORG
entity: the Bay Area	type: LOC


In [3]:
path = "dataset/"

In [4]:
pd.set_option('display.max_columns',None)

In [5]:
vax_symptoms_data = pd.read_csv("VAX_SYMPTOMS_DATA.csv")

In [6]:
vax_vaers_data = pd.read_csv("VAX_VAERS_DATA.csv")

In [7]:
vax_vaers_data.head()

,Unnamed: 0,VAERS_ID,RECVDATE,STATE,AGE_YRS,CAGE_YR,CAGE_MO,SEX,RPT_DATE,SYMPTOM_TEXT,DIED,DATEDIED,L_THREAT,ER_VISIT,HOSPITAL,HOSPDAYS,X_STAY,DISABLE,RECOVD,VAX_DATE,ONSET_DATE,NUMDAYS,LAB_DATA,V_ADMINBY,V_FUNDBY,OTHER_MEDS,CUR_ILL,HISTORY,PRIOR_VAX,SPLTTYPE,FORM_VERS,TODAYS_DATE,BIRTH_DEFECT,OFC_VISIT,ER_ED_VISIT,ALLERGIES
0,0,1217550,04/16/2021,OH,19.0,NaN,NaN,M,NaN,WOKE UP SWEATING; BODY SORENESS; CHILLS; HEADA...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,04/10/2021,NaN,Test Date: 20210411; Test Name: Body temperatu...,OTH,NaN,NaN,NaN,Comments: Unknown,NaN,USJNJFOC20210422234,2,04/14/2021,NaN,NaN,NaN,NaN
1,1,1427815,06/25/2021,NY,76.0,76.0,NaN,F,NaN,I started having bad headaches about two weeks...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,02/27/2021,02/28/2021,1.0,None,OTH,NaN,Ezetimibe; pravastatin; amlodipine besylate; l...,A-fib,A-fib,Influenza Shot,vsafe,2,06/25/2021,NaN,Y,NaN,Pradaxa
2,2,2162583,03/07/2022,MN,89.0,89.0,NaN,M,NaN,Fully vaccinated and boosted on 10/15/21. COV...,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,10/15/2021,03/04/2022,140.0,NaN,UNK,NaN,NaN,NaN,NaN,NaN,NaN,2,03/07/2022,NaN,NaN,NaN,NaN
3,3,2189423,02/24/2022,NY,20.0,20.0,NaN,M,NaN,"On December 23, 2021, my son got a blood draw ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U,07/29/2021,12/23/2021,147.0,NaN,PHM,NaN,None,None,None,NaN,NaN,2,01/23/2022,NaN,Y,NaN,None
4,4,2422917,08/29/2022,FL,67.0,67.0,NaN,M,NaN,patient hospitalized,NaN,NaN,NaN,NaN,Y,13.0,NaN,NaN,U,09/16/2021,08/16/2022,334.0,NaN,PVT,NaN,NaN,NaN,NaN,NaN,NaN,2,08/29/2022,NaN,NaN,NaN,NaN


In [8]:
vax_vaers_data['SYMPTOM_TEXT']

0       WOKE UP SWEATING; BODY SORENESS; CHILLS; HEADA...
1       I started having bad headaches about two weeks...
2       Fully vaccinated and boosted on 10/15/21.  COV...
3       On December 23, 2021, my son got a blood draw ...
4                                    patient hospitalized
                              ...                        
9168    state registry says 15 month old patient recei...
9169    Have not been having regular periods anymore; ...
9170    Extremely sore arm Chills Slight headache Body...
9171    Soreness in my shoulder; This spontaneous case...
9172    INAPPROPRIATE ROUTE OF VACCINE ADMINISTRATION;...
Name: SYMPTOM_TEXT, Length: 9173, dtype: object

In [10]:
stanza.download('en', package='mimic', processors={'ner': 'i2b2'})

2023-11-30 11:45:21 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package        |
------------------------------------
| tokenize        | mimic          |
| pos             | mimic_charlm   |
| lemma           | mimic_nocharlm |
| depparse        | mimic_charlm   |
| ner             | i2b2           |
| backward_charlm | mimic          |
| forward_charlm  | mimic          |
| pretrain        | mimic          |

2023-11-30 11:45:22 INFO: File exists: C:\Users\aishw\stanza_resources\en\tokenize\mimic.pt
2023-11-30 11:45:22 INFO: File exists: C:\Users\aishw\stanza_resources\en\pos\mimic_charlm.pt
2023-11-30 11:45:22 INFO: File exists: C:\Users\aishw\stanza_resources\en\lemma\mimic_nocharlm.pt
2023-11-30 11:45:23 INFO: File exists: C:\Users\aishw\stanza_resources\en\depparse\mimic_charlm.pt
2023-11-30 11:45:24 INFO: File exists: C:\Users\aishw\stanza_resources\en\ner\i2b2.pt
2023-11-30 11:45:24 INFO: File exists: C:\Users\aishw\stanza_resou

In [11]:
nlp_pipe = stanza.Pipeline('en', package='mimic', processors={'ner': 'i2b2'})

2023-11-30 11:45:24 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-11-30 11:45:26 INFO: Loading these models for language: en (English):
| Processor | Package        |
------------------------------
| tokenize  | mimic          |
| pos       | mimic_charlm   |
| lemma     | mimic_nocharlm |
| depparse  | mimic_charlm   |
| ner       | i2b2           |

2023-11-30 11:45:26 INFO: Using device: cpu
2023-11-30 11:45:26 INFO: Loading: tokenize
2023-11-30 11:45:26 INFO: Loading: pos
2023-11-30 11:45:26 INFO: Loading: lemma
2023-11-30 11:45:26 INFO: Loading: depparse
2023-11-30 11:45:27 INFO: Loading: ner
2023-11-30 11:45:27 INFO: Done loading processors!


In [9]:
vax_vaers_data['SYMPTOM_TEXT'][0]

"WOKE UP SWEATING; BODY SORENESS; CHILLS; HEADACHE; FEVER; THREW UP ABOUT EIGHT TIMES; TROUBLE BREATHING; This spontaneous report received from a parent concerned a 19 year old male. The patient's weight, height, and medical history were not reported. The patient received covid-19 vaccine ad26.cov2.s (suspension for injection, route of admin not reported, batch number: Unknown) dose was not reported, administered on 10-APR-2021 for prophylactic vaccination. The batch number was not reported and it has been requested. No concomitant medications were reported.  On 10-APR-2021, the subject experienced threw up about eight times. On 10-APR-2021, the subject experienced trouble breathing. On 10-APR-2021, the subject experienced body soreness. On 10-APR-2021, the subject experienced chills. On 10-APR-2021, the subject experienced headache. On 10-APR-2021, the subject experienced fever. On 11-APR-2021, the subject experienced woke up sweating.  Laboratory data included: Body temperature (NR: 

In [15]:
doc = nlp_pipe(vax_vaers_data['SYMPTOM_TEXT'][0])

In [17]:
for ent in doc.entities:
    print(f'{ent.text}\t{ent.type}')

SWEATING	PROBLEM
BODY SORENESS	PROBLEM
CHILLS	PROBLEM
HEADACHE	PROBLEM
FEVER	PROBLEM
TROUBLE BREATHING	PROBLEM
The patient's weight	TEST
height	TEST
covid-19 vaccine ad26.cov2.s	TREATMENT
admin	TREATMENT
prophylactic vaccination	TREATMENT
The batch number	TEST
concomitant medications	TREATMENT
trouble breathing	PROBLEM
body soreness	PROBLEM
chills	PROBLEM
headache	PROBLEM
fever	PROBLEM
sweating	PROBLEM
Laboratory data	TEST
Body temperature	TEST
covid-19 vaccine ad26.cov2.s	TREATMENT
trouble breathing	PROBLEM
chills	PROBLEM
fever	PROBLEM
sweating	PROBLEM
body soreness	PROBLEM
headache	PROBLEM


In [45]:
def symptom_extractor(sentence):
    doc = nlp_pipe(sentence)
    symptoms = []
    for ent in doc.entities:
        if ent.type=='PROBLEM':
#             print(f'{ent.text}')
            symptoms.append(ent.text)
#     print(symptoms)
    return symptoms

In [48]:
symptoms_list = []
for i in tqdm(vax_vaers_data['SYMPTOM_TEXT'].dropna(), total=len(vax_vaers_data['SYMPTOM_TEXT'].dropna()), leave=True):
    symptoms_list.extend(symptom_extractor(i))
#     break

100%|██████████| 9162/9162 [7:22:01<00:00,  2.89s/it]   


In [32]:
symptoms_list

['SWEATING',
 'BODY SORENESS',
 'CHILLS',
 'HEADACHE',
 'FEVER',
 'TROUBLE BREATHING',
 'trouble breathing',
 'body soreness',
 'chills',
 'headache',
 'fever',
 'sweating',
 'trouble breathing',
 'chills',
 'fever',
 'sweating',
 'body soreness',
 'headache']

In [52]:
import json
with open("extracted_symptoms_list.json",'w') as op_file:
    json.dump(list(symptoms_list),op_file)